In [57]:
import pandas as pd
import os
import numpy as np

In [58]:
filePath = './data/raw/'
filelist=os.listdir(filePath)
filelist


['08A85D3DA29137032D684589013275CFki.tsv']

In [63]:
for name in filelist:
    filepath_new=filePath+name
    print(filepath_new)
    file=pd.read_csv(filepath_new,sep='\t', header=0)
    #extract the ki,ic50,kd,ec50 of data.
    extract_data = file[['Ligand SMILES', 'Ki (nM)', 'IC50 (nM)', 'Kd (nM)', 'EC50 (nM)', 'BindingDB Target Chain Sequence']].copy()
    extract_data['combined'] = extract_data.apply(lambda row: row['Ki (nM)'] if pd.notna(row['Ki (nM)']) else 
                                                            (row['IC50 (nM)'] if pd.notna(row['IC50 (nM)']) else 
                                                            (row['Kd (nM)'] if pd.notna(row['Kd (nM)']) else row['EC50 (nM)'])), axis=1)
    print(extract_data)
    csv_path='./data/extract_data/'+name+'.csv'
    extract_data.to_csv(csv_path,index=False,header=True)
    affinity_data=extract_data[['Ligand SMILES','BindingDB Target Chain Sequence','combined']].copy()
    print(affinity_data)

    # Remove the '<' and '>' symbols from the 'combined' columns.
    affinity_data['combined'] = affinity_data['combined'].replace({'<': '', '>': ''}, regex=True)

    # Convert the data in the 'combined' column to float, multiply by 10^-9
    affinity_data['combined'] = affinity_data['combined'].astype(float) * 1e-9


    # check if 'combined' is positive
    nonpositive_values = affinity_data['combined'] <= 0
    if nonpositive_values.any():
        print("Warning: Non-positive values will be ignored in -log10 calculation.")

    # calculate -log10(),ignore the nonpositive values.If the value is not positive, the result is set to NaN
    affinity_data['Affinity'] = np.where(
        affinity_data['combined'] > 0,
        -np.log10(affinity_data['combined']),
        np.nan 
    )
    affinity_data=affinity_data[['Ligand SMILES','BindingDB Target Chain Sequence','Affinity']].copy()
    print(affinity_data)
    csv_path=csv_path='./data/Affinity/'+name+'.csv'
    affinity_data.to_csv(csv_path,index=False,header=True)

./data/raw/08A85D3DA29137032D684589013275CFki.tsv
                                         Ligand SMILES  Ki (nM) IC50 (nM)  \
0                     Nc1cc2cc(ncc2cc1I)-n1ccc2ccncc12    0.180       NaN   
1                      Fc1ccc2cnc(cc2c1)-n1ccc2ccncc12    0.220       NaN   
2                   Nc1cc(F)cc2cnc(cc12)-n1ccc2ccncc12    0.224       NaN   
3                     COc1ccc2cc(ncc2c1)-n1ccc2ccncc12    0.290       NaN   
4                      Nc1ccc2cc(ncc2c1)-n1ccc2ccncc12    0.290       NaN   
..                                                 ...      ...       ...   
477  COc1cc(\C=C\c2cc(\C=C\c3ccc(O)c(OC)c3)n(n2)-c2...      NaN     35000   
478  COc1cc(\C=C\c2cc(\C=C\c3ccc(O)c(OC)c3)n(n2)-c2...      NaN     54000   
479  COc1cc(\C=C\c2cc(\C=C\c3ccc(O)c(OC)c3)n(CCO)n2...      NaN     69000   
480       CC(=O)N1N=C(CC1c1ccc(O)c(O)c1)c1ccc(O)c(O)c1      NaN     78510   
481  COc1cc(\C=C\c2cc(\C=C\c3ccc(O)c(OC)c3)n(n2)-c2...      NaN    103000   

     Kd (nM)  EC50 (nM)  